In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd

import h5py
import seaborn as sns
from sklearn.metrics import mean_squared_error,  mean_absolute_error, r2_score						 
from scipy.stats import pearsonr

plt.rcParams["font.family"] = "Times New Roman" 

# Model evaluation
- Median wind speeds
- Time series
- Probability distribution
- Quantiles

In [ ]:
#	Define the model column name
PREDCOLS = ['UC-ERA5', 'TI-GWA3', 'TI-GBOOST', 'TR-LSTM', 'TR-Transformer']

DATE_ORIGINE = pd.Timestamp('1900-01-01 00:00')

#	Define some additional metrics
pearson = lambda x, y: pearsonr(x, y)[0]
root_mean_squared_error = lambda o, p: np.sqrt(mean_squared_error(o, p))
mean_error = lambda o, p: np.mean(p - o)



In [ ]:
#	Path to the data
LSTMPath = r'../Predictions/LSTM/train_test/test.npy' 
TransformerPath = r'../Predictions/Transformer/train_test/test.npy' 
GBPATH = r'../Predictions/TI_GBOOST/test.csv' 

OBSPATH = r'../data/ECCC_PARQUET'
StaticPath = r'../data/staticFeatures.csv'
TRAINPATH = r'../data/TRAIN_TEST/train.csv'



In [ ]:
#	Load the static data to get the type of environment of each station
Static = (pd.read_csv(StaticPath, index_col='Climate_ID'))
#	Load Canada shapefile with the provinces
Canada_shp = gpd.read_file(r'../data/Canada_shpfile/Canada_provinces.shp')

In [ ]:
#	Load the npy files for the LSTM and Transformer test predictions
LSTM_Y_hat = np.load(LSTMPath)
Transformer_Y_hat = np.load(TransformerPath)

#	Put the predictions in a Pandas DataFrame with the time and station ID
srcPath = r'../data/H5dataset/train_test/test.h5'
with h5py.File(srcPath, 'r') as f:
	
	Climate_ID = np.char.decode(f['Climate_ID'][:], 'utf-8')
	dates = f['Dates'][:]
	dates = pd.to_datetime(dates, unit='h', origin=DATE_ORIGINE)
	Index = pd.MultiIndex.from_frame(dates.to_frame(name='date').assign(Climate_ID=Climate_ID))

columns = np.arange(LSTM_Y_hat.shape[-1])

LSTM_ratio_pred =  pd.DataFrame(LSTM_Y_hat, index=Index, columns=columns)
LSTM_ratio_pred = (LSTM_ratio_pred.melt(var_name='Timedelta', value_name='LSTM_ratio', ignore_index=False)
					.assign(Date_UTC = lambda x: pd.to_datetime(x.index.get_level_values('date')) +
														pd.TimedeltaIndex(x['Timedelta'].astype(int), 'h'))
					.drop(columns='Timedelta')
					.reset_index().set_index(['Climate_ID', 'Date_UTC'])
														)


Transformer_ratio_pred =  pd.DataFrame(Transformer_Y_hat, index=Index, columns=columns)
Transformer_ratio_pred = (Transformer_ratio_pred.melt(var_name='Timedelta', value_name='Transformer_ratio', ignore_index=False)
					.assign(Date_UTC = lambda x: pd.to_datetime(x.index.get_level_values('date')) +
														pd.TimedeltaIndex(x['Timedelta'].astype(int), 'h'))
					.drop(columns='Timedelta')
					.reset_index().set_index(['Climate_ID', 'Date_UTC'])
														)



In [ ]:
Climate_ID = Transformer_ratio_pred.index.get_level_values('Climate_ID').unique()
years = Transformer_ratio_pred.index.get_level_values('Date_UTC').year.unique()

#  load the observations
obs_era = (pd.read_parquet(OBSPATH, 
					columns=['WS_MS', 'ERA5_WS_MS'], 
					filters=[('Climate_ID', 'in', Climate_ID), 
								('Year', 'in', years)])
					.sort_index()
		)


#  Compute the GWA scaling factor
GWA3_ratio = (pd.read_csv(StaticPath, index_col='Climate_ID')
			  .assign(gwa_ratio = lambda x : (x.GWA_10ws / x['10ws_mean']))
			  .loc[Climate_ID, ['gwa_ratio']])

#  load the TI-G
GB_ratio_pred = (pd.read_csv(GBPATH, index_col='Climate_ID')
                  	.rename(columns={'predictions':'ratio_GB'})
					.loc[Climate_ID, 'ratio_GB']
					)

#  merge all the data predictions and compute corrected ERA5 wind speeds

data = (	LSTM_ratio_pred
				.merge(Transformer_ratio_pred, left_index=True, right_index=True)
				
				.merge(obs_era, left_index=True, right_index=True, how='inner')
				.dropna(subset=['WS_MS'])
                .merge(GB_ratio_pred, left_on='Climate_ID', right_index=True)
				.merge(GWA3_ratio, left_on='Climate_ID', right_index=True)
				.assign(obs_ratio = lambda x: x.WS_MS / x.ERA5_WS_MS)
				.assign(TI_GWA3 = lambda x: (x.ERA5_WS_MS * x.gwa_ratio) )
				.assign(TR_Transformer = lambda x: (x.Transformer_ratio * x.ERA5_WS_MS) )
                .assign(TR_LSTM = lambda x: (x.LSTM_ratio * x.ERA5_WS_MS) )
				.assign(TI_GBOOST = lambda x: x.ERA5_WS_MS * x.ratio_GB)
				.rename(columns={'TI_GBOOST':'TI-GBOOST', 
                     				'TI_GWA3':'TI-GWA3', 
                                    'TR_LSTM':'TR-LSTM', 
                                    'TR_Transformer':'TR-Transformer',
                                    'ERA5_WS_MS': 'UC-ERA5'
                                    })
		)

## Median wind speed evaluation

In [ ]:
metrics =  {
			'RMSE': root_mean_squared_error,
			'MAE': mean_absolute_error,
			'R2': r2_score,
			'MBE':mean_error
								}


station_median = (data.groupby(['Climate_ID'])[PREDCOLS + ['WS_MS']]
						.median()
						.assign(Area = lambda x: Static.loc[x.index, 'Type_of_location'])
				)

ABS_prfm =  {}
ABS_prfm_overall = {}
for name, metric in metrics.items():

	abs_prfm_overall = pd.Series({col: metric(station_median.WS_MS, station_median[col]) for col in PREDCOLS})

	abs_prfm = (pd.concat([station_median.groupby('Area')
							.apply(lambda x: metric(x.WS_MS, x[col]))
							.to_frame(col)
						for col in PREDCOLS],
						axis=1)
						)
					

	ABS_prfm[name] = abs_prfm
	ABS_prfm_overall[name] = abs_prfm_overall
median_eval = pd.concat(ABS_prfm, names=['Metric'])
median_eval_overall = pd.concat(ABS_prfm_overall, names=['Metric'])

In [ ]:
median_eval_overall_pivot = (median_eval_overall
										.to_frame('value')
										.reset_index()
										.rename(columns={'level_1': 'Model'})
										.assign(Area='overall')
										.pivot(index=['Metric', 'Model'], columns='Area', values='value')
                                   )
median_eval_pivot = (median_eval
							.melt(ignore_index=False, var_name='Model')
							.reset_index()
							.pivot(index=['Metric', 'Model'], columns='Area', values='value'))
median_perfomance = pd.concat([median_eval_pivot, median_eval_overall_pivot], axis=1)

In [ ]:
(median_perfomance
	.melt(ignore_index=False, var_name='Area')
	.reset_index()
	.pivot(index=['Metric', 'Area'], columns=['Model'], values='value', )
	.round(2)
	.loc[:, PREDCOLS]
	
 
 )

### Scatter plot between observed and predicted median wind speeds

In [ ]:
PC_CLASSES = sorted(station_median.Area.unique())

fig, axes = plt.subplots(len(PREDCOLS) , len(PC_CLASSES), figsize=(15, 12), sharex=True, sharey=True, gridspec_kw = {'wspace':0.1, 'hspace':0.1})
for i, (PC_class, axs) in enumerate(zip(PC_CLASSES, axes.T)):
    for j, (col, ax) in enumerate(zip(PREDCOLS, axs)):
        d = station_median.loc[station_median.Area == PC_class]
        
        ax.scatter(d.loc[:, 'WS_MS'],
                    d.loc[:, col],
                    alpha=0.4,
                    label=col,
                    edgecolor='w',
                
                    )
        
        ax.axline([0, 0], [1, 1], ls='--', color='k', zorder=0, lw=1.5, alpha=0.5)
        
        ax.set_xticks(np.arange(0, 12, 2), np.arange(0, 12, 2))
        ax.set_yticks(np.arange(0, 12, 2), np.arange(0, 12, 2))
        ax.set_xlim(0, 11)
        ax.set_ylim(0, 11)
        
        if j == 0:
            ax.set_title(f'{PC_class}', fontsize=15)

        if j == len(PREDCOLS) - 1:
            ax.set_xlabel('Observed \nmedian wind speed (m/s)', fontsize=15)
        if i == 0:
            ax.set_ylabel(f'{col} (m/s)', fontsize=15)

        ax.spines[['left', 'right', 'top', 'bottom']].set_alpha(0.1)
        ax.tick_params(axis='both',  direction='in', labelsize=12, top = False, left=False, right=False, bottom=False, color='w')
        ax.grid(alpha=0.5, clip_on=True, axis='y', which='major', ls='--')


## Time series evaluation

In [ ]:
metrics =  {
			'RMSE': root_mean_squared_error,
			'MAE': mean_absolute_error,
			'PCC': pearson,
			'MBE':mean_error
								}

metric_perfect = {'RMSE':  0,'MAE': 0,'PCC': 1, 'MBE':0 }

ABS_prfm, REL_prfm = {}, {}


#Compute performance for each metric
for name, metric in metrics.items():

	abs_prfm = (pd.concat([data.groupby(['Climate_ID'])
							.apply(lambda x: metric(x.WS_MS, x[opt]))
							.to_frame(opt)
								for opt in PREDCOLS],
						axis=1)
						.assign(Area = lambda x: Static.loc[x.index, 'Type_of_location'])
						)
	
	rel_prfm = (abs_prfm.set_index('Area', append=True)
			 			.transform(lambda x: ((x - x['UC-ERA5'])/(metric_perfect[name] - x['UC-ERA5'])), axis=1)
						.reset_index(level='Area')
						)

	
	ABS_prfm[name] = abs_prfm
	REL_prfm[name] = rel_prfm

	absolute_preformance = pd.concat(ABS_prfm, names=['Metric']).set_index('Area', append=True)
	relative_preformance = pd.concat(REL_prfm, names=['Metric']).set_index('Area', append=True)

	absolute_preformance_mean = absolute_preformance.groupby(['Metric', 'Area']).mean().round(2)
	relative_preformance_mean = relative_preformance.groupby(['Metric', 'Area']).mean().round(2)


TS_eval = absolute_preformance.melt(ignore_index=False, var_name='Model', value_name='value').reset_index()
TS_eval_rel = relative_preformance.drop(columns=['UC-ERA5']).melt(ignore_index=False, var_name='Model', value_name='value').reset_index()

In [ ]:
grouped = (relative_preformance
           .groupby(['Metric', 'Area'])
           .median()
           .round(2))

overall = (relative_preformance
           	.groupby(['Metric'])
            .median()
            .round(2)
            .assign(Area='Overall')
            .set_index('Area', append=True)
            )

pd.concat([grouped, overall]).sort_index().loc[lambda x: ~x.index.get_level_values(0).isin(['PCC', 'MBE']), PREDCOLS]

In [ ]:
grouped = (absolute_preformance
           .groupby(['Metric', 'Area'])
           .median()
           .round(2))

overall = (absolute_preformance
           	.groupby(['Metric'])
            .median()
            .round(2)
            .assign(Area='Overall')
            .set_index('Area', append=True)
            )

pd.concat([grouped, overall]).sort_index().loc[:, PREDCOLS]

### Plot the time series metrics
- distribution of time series metrics (RMSE, MAE, PCC, MBE)
- distribution of percentage improvement over ERA5 (RMSE, MAE)
- Map of percentage improvement over ERA5 (RMSE, MAE)

In [ ]:
# plot distribution of time series metrics
metrics =sorted( ['RMSE', 'MBE', 'MAE', 'PCC'])
PC_CLASSES = sorted(TS_eval['Area'].unique())
fig, axes = plt.subplots(len(metrics), len(PC_CLASSES), figsize=(12, 10), sharex=True, sharey='row')

fsz0 = 14

for i, (axs, PC_class) in enumerate(zip(axes.T, PC_CLASSES)):

	for k, (ax, m) in enumerate(zip(axs.flatten(), metrics)):


		d = TS_eval.loc[lambda x: (x.Metric == m) & (x.Area == PC_class)]
		median_era5 = d.loc[lambda x: x.Model == 'UC-ERA5'].value.median()

		ax = sns.boxplot(data=d, 
							x='Model', y='value',  
							ax=ax, hue='Model', log_scale=False, dodge=False, gap=0.1,
							width=0.7,
							legend='brief',
							flierprops={"marker": "x", 'alpha':0.5},
							palette= 'Set2',
							whis=(10, 90),
							showcaps=True,
							showfliers=False,
						)
		if m == 'MBE':
			ax.axhline(0, ls='--', color='g', lw=1.5, alpha=0.5)
		
		ax.axhline(median_era5, ls='-.', color='r', lw=1.5, alpha=0.8)
		
		ax.spines[['left', 'right', 'top', 'bottom']].set_alpha(0.1)
		ax.tick_params(axis='both',  direction='in', labelsize=fsz0, top = False, left=False, right=False, bottom=False, color='w')
	

		ax.legend().remove()
		
		ax.set_xlabel('')
		ax.set_xticklabels('')
		if k == 0:
			ax.set_title(f'{PC_class}', fontsize=fsz0)
		if i == 0:
			unit = '$\mathregular{ms^{-1}}$'
			if m in ['RMSE', 'MAE']:
				ax.set_ylabel(f'{m} ({unit})', fontsize=fsz0)
			else:
				ax.set_ylabel(m, fontsize=fsz0)
		else:
			ax.set_ylabel('')
		ax.grid(alpha=0.5, clip_on=True, axis='y', which='major', ls='--')

plt.figlegend(*ax.get_legend_handles_labels(),  ncol=5,  bbox_to_anchor=(0.5, -0.01), loc='upper center', fontsize=fsz0, title_fontproperties={'size': fsz0}, framealpha=0)

plt.tight_layout()


In [ ]:
# Plot distribution of percentage improvement over ERA5
metrics =sorted( ['RMSE', 'MAE'])
PC_CLASSES = sorted(TS_eval['Area'].unique())
fig, axes = plt.subplots(len(metrics), len(PC_CLASSES), figsize=(12, 7), sharex=True, sharey='row')

fsz0 = 12

for i, (axs, PC_class) in enumerate(zip(axes.T, PC_CLASSES)):

	for k, (ax, m) in enumerate(zip(axs.flatten(), metrics)):


		d = TS_eval_rel.loc[lambda x: (x.Metric == m) & (x.Area == PC_class)].assign(value = lambda x: x.value * 100)
		

		ax = sns.boxplot(data=d, 
							x='Model', y='value',  
							ax=ax, hue='Model', log_scale=False, dodge=False, gap=0.1,
							width=0.7,
							legend='brief',
							flierprops={"marker": "x", 'alpha':0.5},
							palette= 'Set2',
							whis=(10, 90),
							showcaps=True,
							showfliers=False,
						)
		
		ax.axhline(0, ls='-.', color='r', lw=1.5, alpha=0.8)
		
		ax.spines[['left', 'right', 'top', 'bottom']].set_alpha(0.1)
		ax.tick_params(axis='both',  direction='in', labelsize=fsz0, top = False, left=False, right=False, bottom=False, color='w')
	

		ax.legend().remove()
		
		ax.set_xlabel('')
		ax.set_xticklabels('')
		if k == 0:
			ax.set_title(f'{PC_class}', fontsize=fsz0)
		if i == 0:
			unit = '$\mathregular{ms^{-1}}$'
			if m in ['RMSE', 'MAE']:
				ax.set_ylabel(f'{m}\n Percentage improvement (%)', fontsize=fsz0)
			else:
				ax.set_ylabel(m, fontsize=fsz0)
		else:
			ax.set_ylabel('')
		ax.grid(alpha=0.5, clip_on=True, axis='y', which='major', ls='--')

plt.figlegend(*ax.get_legend_handles_labels(),  ncol=5,  bbox_to_anchor=(0.5, -0.01), loc='upper center', fontsize=fsz0, title_fontproperties={'size': fsz0}, framealpha=0)

plt.tight_layout()


In [ ]:
#	Plot the map of percentage improvement over ERA5 wind speeds


fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(2, 2, wspace=0.0, hspace=0.0)
axes = gs.subplots(sharey=True, sharex=True)

TS_TR_rel = (TS_eval_rel
			 .assign(value = lambda x: x.value * 100)
				.assign(EPSG3979_x = lambda x: Static.loc[x.Climate_ID, 'X_EPSG3979'].values,
						EPSG3979_y = lambda x: Static.loc[x.Climate_ID, 'Y_EPSG3979'].values)
				.loc[lambda x: x.Model.isin(['TR-LSTM', 'TR-Transformer'])]
				.pivot(index=['Model','Climate_ID', 'EPSG3979_x', 'EPSG3979_y'], columns='Metric', values='value')
		
				)
Metrics = sorted(['RMSE', 'MAE'])
Colors = {'RMSE': 'RdBu', 'MAE': 'RdBu', 'PCC': 'RdBu'}

Models = ['TR-LSTM', 'TR-Transformer']
label = {0:'a)', 1:'b)', 2:'c)', 3:'d)', 4:'e)', 5:'f)'}
fsz0 = 14
for i, (axs, model) in enumerate(zip(axes, Models)):

	for k, (ax, metric) in enumerate(zip(axs, Metrics)):

		data_to_plot = (TS_TR_rel[[metric]]
						.reset_index(level='Model')
						.pivot(columns='Model', values=metric)
						.reset_index(level=['EPSG3979_x', 'EPSG3979_y'])
						)

		ax = Canada_shp.plot(ax=ax, facecolor=(0, 0, 0, 0.05), edgecolor=(0,0,0, 0.4), linewidth=0.1)

		vmin = -10
		vmax = 10
		norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
		cax = ax.scatter(	data_to_plot['EPSG3979_x'], 
							data_to_plot['EPSG3979_y'], 
							c=data_to_plot[model], 
							cmap=Colors[metric], 
							s=20, norm=norm,
							alpha=0.5,
							edgecolors=None,
							)
		
					
		if k == 0:
			ax.set_ylabel(f'{model}', fontsize=fsz0)

		if i == 0:
			ax.set_title(f'{metric}', fontsize=fsz0)

		
		ax.spines[['left', 'right', 'top', 'bottom']].set_alpha(0)
		ax.tick_params(axis='both',  
					direction='in', labelsize=10, 
					top = False, left=False, right=False,
					bottom=False, labelbottom=False, 
					labeltop=False, labelleft=False, 
					labelright=False)
	
cbar = fig.colorbar(cax, ax=axs, orientation='horizontal', shrink=0.5, pad=0)
cbar.ax.tick_params(labelsize=fsz0)
cbar.set_label('Percentage improvement over\n UC-ERA5 (%)', fontsize=fsz0)
cbar.outline.set_linewidth(0)
cbar.ax.tick_params(axis='x',  direction='out', labelsize=fsz0, top = False, left=False, right=False, bottom=True,  length=2, color=(0, 0, 0, 0.5), width =0.5)


## Probability distribution evaluation
- across the entire distribution PSS-ALL
- for the lower tail (PSS-LWT)
- for the upper tail (PSS-UPT)

In [ ]:
data = (data.assign(	OBS_QL = lambda x: x.groupby('Climate_ID').WS_MS.transform(lambda x: x.quantile(0.10)),
						OBS_QU = lambda x: x.groupby('Climate_ID').WS_MS.transform(lambda x: x.quantile(0.90))
					)	
			)

def compute_PSS(data, var_name='PSS'):
	col = ['WS_MS'] + PREDCOLS
	maxall = int(data[col].max().max()) + 1
	minall = int(data[col].min().min()) 
	binedges = np.arange(minall, maxall, 1)
	
	
	PSS = (data[col].transform(lambda x: pd.cut(x, binedges, include_lowest=True))
					.melt(ignore_index=False, var_name='Model', value_name='bin')
					.groupby(['Model', 'Climate_ID'])
					.value_counts(normalize=True)
					.to_frame('prop')
					.reset_index()
					.pivot(index=['Climate_ID', 'bin'], columns='Model', values='prop')
					.fillna(0)
					.transform(lambda x: x.combine(x.WS_MS, min), axis=1)
					.groupby('Climate_ID')
					.sum()
					.assign(Area = lambda x: Static.loc[x.index, 'Type_of_location'])
					.drop(columns=['WS_MS'])
					.reset_index()
					.melt(id_vars=['Climate_ID', 'Area'], 
								value_name=var_name, var_name='Model')
								.set_index(['Climate_ID', 'Area', 'Model'])
				
				)
	return PSS

PSS_L = compute_PSS(data.loc[lambda x: x.WS_MS <= x.OBS_QL], 'PSS-LWT')
PSS_M = compute_PSS(data, 'PSS-ALL')
PSS_H = compute_PSS(data.loc[lambda x: x.WS_MS >= x.OBS_QU], 'PSS-UPT')

PSS = pd.concat([PSS_L, PSS_M, PSS_H], axis=1).mul(100)



PSS_median = (PSS.melt(ignore_index=False, var_name='PSS', value_name='value')
					.reset_index()
					.pivot(index=['Climate_ID', 'Area', 'PSS'], columns='Model', values='value')
					.reset_index()
					.set_index(['Climate_ID'])
					.groupby(['PSS','Area',])
					.median()
					.round(2))

In [ ]:
overall = (PSS.melt(ignore_index=False, var_name='PSS', value_name='value')
					.reset_index()
					.pivot(index=['Climate_ID', 'Area', 'PSS'], columns='Model', values='value')
					.reset_index()
					.set_index(['Climate_ID'])
                    .drop(columns=['Area'])
                    .groupby(['PSS'])
					.median()
					.round(2)
                    .assign(Area='Overall')
                    .set_index('Area', append=True)
                    )

pd.concat([PSS_median, overall]).sort_index().loc[:, PREDCOLS]#.to_clipboard()

###	Plot the probability distribution metrics

In [ ]:
PSS_levels = sorted(['PSS-LWT', 'PSS-ALL', 'PSS-UPT'])

Groups = sorted(PSS.index.get_level_values('Area').unique())


fig, axes = plt.subplots(3, len(Groups), figsize=(12, 10), sharey=True)
fsz0 = 14
for k, (axs, group) in enumerate(zip(axes.T, Groups)):
	for i, (ax, pss_level) in enumerate(zip(axs, PSS_levels)):

		d = PSS.loc[(slice(None), slice(None), PREDCOLS)].loc[lambda x: (x.index.get_level_values('Area') == group), [pss_level]].reset_index()
		median_era5 = d.loc[lambda x: x.Model == 'UC-ERA5', pss_level].median()

		ax = sns.boxplot(data=d, 
							x='Model', y=pss_level,  
							ax=ax, hue='Model', 
							log_scale=False, 
							dodge=False, gap=0.1,
							width=0.7,
							legend='brief',
							flierprops={"marker": "x", 'alpha':0.5},
							palette= 'Set2',
							whis=(10, 90),
							showcaps=True,
							showfliers=False,
						)

		ax.axhline(median_era5, ls='-.', color='r', lw=1.5, alpha=0.8)
		ax.set_title(f'{group}', fontsize=fsz0)
		ax.spines[['left', 'right', 'top', 'bottom']].set_alpha(0.1)
		ax.tick_params(axis='both',  direction='in', labelsize=fsz0, top = False, left=False, right=False, bottom=False, color='w')


		ax.legend().remove()

		ax.set_xlabel('')
		ax.set_xticklabels('')
		ax.set_ylabel(f'{pss_level} (%)', fontsize=fsz0)
		ax.grid(alpha=0.5, clip_on=True, axis='y', which='major', ls='--')

plt.figlegend(*ax.get_legend_handles_labels(),  
			  ncol=len(PREDCOLS),  
			  bbox_to_anchor=(0.5, -0.08), 
			  loc='lower center', 
			  fontsize=fsz0,  
			  title_fontproperties={'size': fsz0}, 
			  framealpha=0.)

plt.tight_layout()


## Quantiles evaluation

In [ ]:
qs = np.arange(0.9, 0, -0.1)
# q_level = pd.Series(q.round(2)).astype(str).str.ljust(4, '0').str.replace('0.', 'P', regex=False)
# qlevel_replace = {'quantile_level': {q[i]: q_level[i] for i in range(len(q))}}

Quantiles = (data
 	.loc[:, PREDCOLS + ['WS_MS']]
    .groupby('Climate_ID')
    .quantile(qs, interpolation='linear')
	.reset_index()
    .rename(columns={'level_1':'Qlevel'})
	.set_index(['Climate_ID'])
	)

In [ ]:
metrics =  {
			'RMSE': root_mean_squared_error,
			'MAE': mean_absolute_error,
			'R2': r2_score,
			'MBE':mean_error
								}


Qeval = {}
for mname, metric in metrics.items():
	qr = []
	for q in qs:

		d = Quantiles.loc[lambda x: x.Qlevel == q]
		qr += [pd.Series({col: metric(d.WS_MS, d[col]) for col in PREDCOLS}, name=f'{q*100:.00f}%')]


					
	qr = pd.DataFrame(qr)
	Qeval[mname] = qr

### Plot the quantile evaluation metrics

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 14), sharex=True, sharey=True, gridspec_kw = dict(wspace=0.2, hspace=0.1))
fsz0 = 20
for mname, ax in zip(metrics.keys(), axes.flatten()):

	annot = Qeval[mname].clip(0).round(2) if mname == 'R2' else Qeval[mname].round(2)
	cax = sns.heatmap(	annot, 
				   		ax=ax, annot=annot, 
						cmap='cividis', fmt='.2f', 

						cbar=False,
						linewidth=1,
						linecolor=(1, 1, 1, 0.5),
				annot_kws = {'fontsize': 20})
	
	ax.tick_params(axis='both',  direction='out', 
					labelsize=fsz0, 
					top = False, 
					left=True, 
					right=False, 
					bottom=True,  
					length=2, 
					color=(0, 0, 0, 0.7), 
					width =0.2,
					labelrotation=45
					)

	cbar = fig.colorbar(cax.collections[0], ax=ax, orientation='vertical', location='right', fraction=0.05, format='%.2f', drawedges=False, pad=0.01)
	cbar.ax.tick_params(labelsize=fsz0, color=(0, 0, 0, 0.7), width =0.2)
	cbar.outline.set_linewidth(0)
	mname = '$\mathregular{R^{2}}$' if mname =='R2' else mname
	cbar.set_label(f'{mname}', fontsize=fsz0)

## Correlation between metrics and static covariates

In [ ]:
Models = ['TR-LSTM', 'TR-Transformer']

Static_Features  = ["10ws_P05",
						"10ws_P50",
						"10ws_P95",
						"GFSG1",
						"SG1-standard_deviation_of_slope",
						"SG10-dev_from_mean_elev",
						"SG10-tangential_curvature",
						"SG100-standard_deviation_of_slope",
						"SG2-standard_deviation_of_slope",
						"SRL_1000m",
						"SRL_500m"]

FEATURES_RENAME = {	'GFSG1':'ELV-100m',
					'SG1-elev_percentile':'ELP-100m', 
					'SG1-dev_from_mean_elev' : 'DME-100m', 
					'SG1-aspect' : 'ASP-100m',
					'SG1-slope' : 'SLP-100m', 
					'SG1-ruggedness_index' : 'RGN-100m',
					'SG1-standard_deviation_of_slope' : 'SDS-100m', 
					'SG1-total_curvature' : 'TOC-100m',
					'SG1-tangential_curvature' : 'TAC-100m', 

					'GFSG2':'ELV-200m',
					'SG2-elev_percentile':'ELP-200m', 
					'SG2-dev_from_mean_elev' : 'DME-200m', 
					'SG2-aspect' : 'ASP-200m',
					'SG2-slope' : 'SLP-200m', 
					'SG2-ruggedness_index' : 'RGN-200m',
					'SG2-standard_deviation_of_slope' : 'SDS-200m', 
					'SG2-total_curvature' : 'TOC-200m',
					'SG2-tangential_curvature' : 'TAC-200m', 

					'GFSG10':'ELV-1km',
					'SG10-elev_percentile':'ELP-1km', 
					'SG10-dev_from_mean_elev' : 'DME-1km', 
					'SG10-aspect' : 'ASP-1km',
					'SG10-slope' : 'SLP-1km', 
					'SG10-ruggedness_index' : 'RGN-1km',
					'SG10-standard_deviation_of_slope' : 'SDS-1km', 
					'SG10-total_curvature' : 'TOC-1km',
					'SG10-tangential_curvature' : 'TAC-1km', 

					'GFSG20':'ELV-2km',
					'SG20-elev_percentile':'ELP-2km', 
					'SG20-dev_from_mean_elev' : 'DME-2km', 
					'SG20-aspect' : 'ASP-2km',
					'SG20-slope' : 'SLP-2km', 
					'SG20-ruggedness_index' : 'RGN-2km',
					'SG20-standard_deviation_of_slope' : 'SDS-2km', 
					'SG20-total_curvature' : 'TOC-2km',
					'SG20-tangential_curvature' : 'TAC-2km',

					'GFSG50':'ELV-5km',
					'SG50-elev_percentile':'ELP-5km', 
					'SG50-dev_from_mean_elev' : 'DME-5km', 
					'SG50-aspect' : 'ASP-5km',
					'SG50-slope' : 'SLP-5km', 
					'SG50-ruggedness_index' : 'RGN-5km',
					'SG50-standard_deviation_of_slope' : 'SDS-5km', 
					'SG50-total_curvature' : 'TOC-5km',
					'SG50-tangential_curvature' : 'TAC-5km',

					'GFSG100':'ELV-10km',
					'SG100-elev_percentile':'ELP-10km', 
					'SG100-dev_from_mean_elev' : 'DME-10km', 
					'SG100-aspect' : 'ASP-10km',
					'SG100-slope' : 'SLP-10km', 
					'SG100-ruggedness_index' : 'RGN-10km',
					'SG100-standard_deviation_of_slope' : 'SDS-10km', 
					'SG100-total_curvature' : 'TOC-10km',
					'SG100-tangential_curvature' : 'TAC-10km',

					'SRL_100m' : 'SRL-100m',
					'SRL_500m' : 'SRL-500m', 
					'SRL_1000m' : 'SRL-1km', 
					'SRL_5.0km' : 'SRL-5km', 
					'SRL_10.0km' : 'SRL-10km', 
					'SRL_20.0km' : 'SRL-20km',
					'Dcoast' : 'Dcoast', 
					'10ws_P05' : 'ERA5-P5%', 
					'10ws_P50' : 'ERA5-P50%', 
					'10ws_P95' : 'ERA5-P95%', 
        }


METRICS = ['MAE',  'RMSE']

metrics_perfect = {'MAE': 0, 'PCC': 1, 'RMSE': 0, 'PSS-ALL': 100, 'PSS-LWT': 100, 'PSS-UPT': 100}

data_to_plot = (TS_eval.pivot(index=['Climate_ID', 'Area', 'Model',], columns='Metric', values='value')
						.merge(PSS, left_index=True, right_index=True)
						.melt(ignore_index=False, var_name='Metric', value_name='value')
						.reset_index()
						)

fig, axes = plt.subplots(1, 2, figsize=(8, 7), sharex=True, sharey=True)
fsz0 = 14
for i, (model, ax) in enumerate(zip(Models, axes)):
	TS_eval_STATIC_FEATURES = pd.concat([(data_to_plot
											.loc[lambda x: x.Metric == metric]
											.set_index(['Climate_ID', 'Area'])
											.pivot(columns='Model', values='value')
											.transform(lambda x: (x - x['UC-ERA5'])/(metrics_perfect[metric] - x['UC-ERA5']), axis=1)
											.loc[:, [model]]
											.rename(columns={model:metric})
											.reset_index()
											.merge(Static[Static_Features], left_on='Climate_ID', right_index=True)
											.set_index(['Climate_ID', 'Area'])
											.corr(method='spearman')
											.loc[lambda x: x.index.isin(Static_Features), metric]
											.rename(index=FEATURES_RENAME)
											.sort_index()
							)
							for metric in METRICS]
							, axis=1, keys=METRICS)
							

	ax.tick_params(axis='both',  direction='out', labelsize=fsz0, top = False, left=False, right=False, bottom=False,  length=5, color=(0, 0, 0, 0.1), width =1)

	cax = sns.heatmap(TS_eval_STATIC_FEATURES, 
				   		ax=ax, 
						annot=TS_eval_STATIC_FEATURES.round(2), 
						cmap='RdBu', 
						center=0, 
						fmt='.2f', 
						cbar=False,
						linewidth=1,
						linecolor=(1, 1, 1, 0.5),
			 annot_kws = {'fontsize': fsz0})

	ax.set_title(f'{model}', fontsize=fsz0)
cbar = fig.colorbar(cax.collections[0], ax=axes, orientation='vertical',  fraction=0.05, format='%.2f', drawedges=False, anchor=(0.5, 10))

cbar.ax.tick_params(labelsize=fsz0)
cbar.ax.tick_params(axis='x',  direction='out', labelsize=fsz0, top = False, left=False, right=False, bottom=True,  length=2, color=(0, 0, 0, 0.5), width =0.5)
cbar.outline.set_linewidth(0)
cbar.set_label('Spearman correlation \ncoefficient', fontsize=fsz0)
